In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pytz
from datetime import datetime
import MetaTrader5 as mt5


## Get Data from MT5

In [ ]:
# -------------------------------------------------------------------------------------------
def fetch_MT5_data(symbol: str, timeframe: int, date_from: datetime, date_to: datetime):
    if not mt5.initialize():
        print(f"initialize() failed, error code = {mt5.last_error()}")
        mt5.shutdown()
    else:
        
        rates = mt5.copy_rates_range(symbol, timeframe, date_from, date_to)
        
        if rates is not None and len(rates) > 0:
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df.to_csv(f'{timeframe} ({date_from} -- {date_to}).csv'.replace(" 00:00:00+00:00", ""), index=False)
        else:
            print(f"Failed to fetch data or no data found. Error: {mt5.last_error()}")
            
        mt5.shutdown()


symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M5

timezone  = pytz.timezone("UTC")
date_from = datetime(2025, 10, 1, tzinfo=timezone)
date_to   = datetime(2025, 12, 1, tzinfo=timezone)

# fetch_MT5_data(symbol, timeframe, date_from, date_to)

In [4]:
mt5.initialize()
tick = mt5.symbol_info_tick("EURUSD")

print("Bid:", tick.bid)
print("Ask:", tick.ask)
print("Last:", tick.last)
print("Volume:", tick.volume)

mt5.shutdown()


Bid: 1.17269
Ask: 1.17271
Last: 0.0
Volume: 0


True